In [1]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [2]:
from transformers import AutoTokenizer, AutoModel
from tokenizers import Tokenizer
from transformers import GPT2LMHeadModel, AutoConfig,GPT2Tokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding

2024-10-20 19:37:51.434755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-20 19:37:51.449770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 19:37:51.465504: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 19:37:51.470240: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 19:37:51.483750: I tensorflow/core/platform/cpu_feature_guar

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/dna_eng_bpe")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
model = AutoModelForSequenceClassification.from_pretrained('dnagpt/gpt_dna_eng_v1', num_labels=3)
model.config.pad_token_id = model.config.eos_token_id

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/497M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at dnagpt/gpt_dna_eng_v1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

In [6]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("dnagpt/dna_splice_site_prediction")['train'].train_test_split(test_size=0.1)

README.md:   0%|          | 0.00/314 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.80M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45619 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label'],
        num_rows: 41057
    })
    test: Dataset({
        features: ['sequence', 'label'],
        num_rows: 4562
    })
})

In [8]:
dataset['train'][0]

{'sequence': 'TTCATACTGGAGAGAAACCTTACAAGTGTGAAGAATGCGGCAAAGCGTTTATATGGTCCTCAAACCTTACTGAACATAAGAAAATTCATACTAGAGAGAAACCCTACAAATGTGAAGAATGTAGTAAAGCATTTAGCCGATCCTCAGCCCTAACTACACATAAGAGGATGCACACTGGAGAGAAACCCTACAAATGTGAAGAATGTGGCAAAGCTTTTAGCCAGTCCTCAACCCTTACTGCACATAAGATAATTCATACTGGAGAGAAACCCTACAAATGTGAAGAATGTGGCAAAGCATTTATATTGTCCTCGACCCTATCTAAACATAAGAGGATTCACACTGGAGAGAAACCCTACAAATGTGAAGAATGTGGCAAAACTTTTAATCAATCCTCAAA',
 'label': 2}

In [9]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  TTCATAC TGGAG AGAAACC TTACAAG TGTGAAG AATGC GGC AAAGCG TTTAT ATGGTCC TCAAACC TTAC TGAAC ATAAG AAAATTC ATACTAG AGAGAA ACCCTAC AAATG TGAAG AATG TAGTAA AGCATT TAGCCG ATCC TCAGCCC TAAC TACAC ATAAG AGGATGC ACAC TGGAG AGAAACCC TACAAATG TGAAG AATGTGGC AAAGC TTTTAGCC AGTCC TCAACCC TTAC TGCAC ATAAG ATAATTC ATAC TGGAG AGAAACCC TACAAATG TGAAG AATGTGGC AAAGC ATTTAT ATTGTCC TCG ACCCTATC TAAAC ATAAG AGGATTC ACAC TGGAG AGAAACCC TACAAATG TGAAG AATGTGGC AAAAC TTTTAATC AATCC TCAA A


In [10]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][0]['sequence'])))

# tokens:  69


In [11]:
# 2. tokenize
def tokenize_function(examples):
    examples['label'] = [int(item) for item in examples['label']]
    return tokenizer(examples['sequence'], truncation=True, padding='max_length', max_length=1024)

# 3. 对数据集应用分词函数
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. 创建一个数据收集器，用于动态填充和遮蔽
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/41057 [00:00<?, ? examples/s]

Map:   0%|          | 0/4562 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sequence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 41057
    })
    test: Dataset({
        features: ['sequence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4562
    })
})

In [14]:
from transformers import TrainingArguments, Trainer
import numpy as np



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="ds_job_dna_eng_2",
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.863500,0.648203,0.697501
2,0.562500,0.585912,0.740465
3,0.398500,0.523836,0.784086
4,0.281500,0.504741,0.811705
5,0.194800,0.574049,0.814336
6,0.140300,0.642204,0.814117
7,0.104300,0.851224,0.810390
8,0.089200,0.904668,0.824638
9,0.077000,1.071279,0.823323
10,0.070700,1.076569,0.824419


TrainOutput(global_step=10270, training_loss=0.2782216949611467, metrics={'train_runtime': 18571.4314, 'train_samples_per_second': 22.108, 'train_steps_per_second': 0.553, 'total_flos': 2.1456315109343232e+17, 'train_loss': 0.2782216949611467, 'epoch': 10.0})